In [1]:
import os

from llama_index.llms.azure_openai import AzureOpenAI

llmmodelAzure = { "model": "gpt-4o",
                "version":os.environ.get('AZURE_OPENAI_VERSION',''),
                "azure_deployment":"gpt-4o",
                "azure_endpoint":os.environ.get('AZURE_ENDPOINT',''),
                "api_key":os.environ.get('AZURE_OPENAI_KEY','')}

llm=AzureOpenAI(azure_deployment=llmmodelAzure["azure_deployment"],
                azure_endpoint=llmmodelAzure["azure_endpoint"],temperature=0.0,
                api_version=llmmodelAzure["version"], api_key=llmmodelAzure["api_key"],
                timeout=120,max_retries=3,top_p=0.0001)

In [2]:
#question = """Pasūtītājs paredz elektronisku pieteikumu un piedāvājumu iesniegšanu. 
#VAI arī ir paskaidrots kāpēc nē ar atsauci uz  MK noteikumu 231.punktu.
#"""
question = """
šis ir jautājums par nosacījumiem pretendenta dalībai konkursā, par piedāvājuma noraidīšanas iespējām. vai pasūtītājs atklātā vai slēgtā konkursā ir paredzējis piemērot PIL 41.panta vienpadsmitās daļas 1.punkta regulējumu?
"""


In [3]:
law = """
Publiskā iepirkuma likums. 41. pants.
(11) Pasūtītājs noraida pretendenta piedāvājumu atklātā vai slēgtā konkursā, ja pretendenta piedāvātā līgumcena pārsniedz jebkuru no šādām vērtībām:

1) iepirkuma procedūras dokumentos pasūtītāja norādīto līgumcenu, ja tā noteikta kā piedāvājuma atbilstības prasība;

2) 150 procentus no iepirkuma procedūras dokumentos norādītās paredzamās līgumcenas.

(12) Pasūtītājs pieņem lēmumu pārtraukt iepirkuma procedūru, ja atklātā konkursā, slēgtā konkursā vai konkursa procedūrā ar sarunām pieteikumu vai piedāvājumu ir iesniedzis tikai viens kandidāts vai pretendents, izņemot gadījumus, kad:

1) pasūtītājs pirms iepirkuma procedūras izsludināšanas ir rīkojis šā likuma 18. panta 2.1 daļas prasībām atbilstošu apspriedi ar piegādātājiem;

2) iepirkuma procedūras pārtraukšana apdraud sabiedrības drošības vai veselības aizsardzības intereses. Šādā gadījumā pasūtītājs iepirkuma procedūras ziņojumā ietver pamatojumu tam, kādas sabiedrības drošības un veselības aizsardzības intereses tiktu apdraudētas, rīkojot atkārtotu iepirkumu.

"""

In [46]:
context = """
3.	NOSACĪJUMI PRETENDENTA DALĪBAI IEPIRKUMĀ
3.1.	Iepirkumā var piedalīties piegādātāji Publisko iepirkumu likuma (turpmāk – PIL) 1.panta 22.punkta izpratnē, kuri ir iesnieguši Nolikumā minētos dokumentus. Piedalīšanās iepirkumā ir Pretendenta brīvas gribas izpausme. Iepirkuma noteikumi visiem Pretendentiem ir vienādi.
3.2.	Pasūtītājs Pretendentu izslēdz no dalības iepirkuma procedūrā jebkurā no PIL 42.panta pirmajā daļā noteiktajiem gadījumiem. Komisija Pretendentu izslēgšanas gadījumus pārbauda PIL 42.panta noteiktajā kārtībā.
3.3.	Saskaņā ar PIL 42.panta četrpadsmito daļu, Pasūtītājs pārbaudi par PIL 42. panta pirmajā daļā noteikto pretendentu izslēgšanas gadījumiem atklātā konkursā veic attiecībā uz katru pretendentu, kuram būtu piešķiramas līguma slēgšanas tiesības.
3.4.	Saskaņā ar PIL 43.panta otro daļu, ja Pretendents vai personālsabiedrības biedrs, ja Pretendents ir personālsabiedrība, atbilst PIL 42.panta pirmās daļas 1., 3., 4., 5., 6. vai 7. punktā minētajam izslēgšanas gadījumam, pretendents norāda to piedāvājumā un, ja tiek atzīts par tādu, kuram būtu piešķiramas līguma slēgšanas tiesības, iesniedz skaidrojumu un pierādījumus par nodarītā kaitējuma atlīdzināšanu vai noslēgtu vienošanos par nodarītā kaitējuma atlīdzināšanu, sadarbošanos ar izmeklēšanas iestādēm un veiktajiem tehniskajiem, organizatoriskajiem vai personālvadības pasākumiem, lai pierādītu savu uzticamību un novērstu tādu pašu un līdzīgu gadījumu atkārtošanos nākotnē.
3.5.	Pirms lēmuma pieņemšanas par līguma slēgšanas tiesību piešķiršanu, iepirkuma komisija attiecībā uz Pretendentu, kuram būtu piešķiramas līguma slēgšanas tiesības, veic pārbaudi par Starptautisko un Latvijas Republikas nacionālo sankciju likuma 11.1 panta pirmajā daļā minēto izslēgšanas gadījumu esamību Starptautisko un Latvijas Republikas nacionālo sankciju likuma 11.1 pantā noteiktajā kārtībā.
3.6.	Pasūtītājs noraida Pretendenta piedāvājumu iepirkuma procedūrā, ja tiek konstatēts PIL 41.panta vienpadsmitās daļas 2.punktā minētie nosacījumi.

"""

In [52]:
prompt = """
Tu esi aģents, kas pārbauda iepirkuma atbilstību likumiem un iepirkuma dokumentācijai. Tev ir pieejami šādi resursi: - Fragmenti no likumiem (<law> tagos), - Fragmenti no iepirkuma dokumentācijas (<context> tagos), - Lietotāja jautājums vai apgalvojums (var būt vairāki teikumi). Tavs uzdevums: 1. Pārbaudi, vai apgalvojums ir patiess, balstoties uz pieejamajiem materiāliem. 2. Izmanto fragmentus no <law> un <context> tagiem, bet drīksti izmantot arī citu sniegto informāciju, ja tāda ir. 3. Ja kontekstā nav pietiekamas informācijas, atbilde ir "nē". Rezultātu atgriez **JSON formātā** ar šādiem laukiem: ```json {  "answer": "jā" | "nē", "rate": "zema" | "vidēja" | "augsta", "explanation": "Īss, bet precīzs paskaidrojums. Norādi, uz kādiem punktiem balstās atbilde — citē attiecīgās vietas no iepirkuma dokumentācijas vai likumiem. Ja atbildes pamatā ir informācijas trūkums, to skaidri norādi." }
"""

_prompt = """Tu esi aģents, kas pārbauda iepirkuma atbilstību likumiem.
Tev ir pieejami fragmenti no likumiem (iekļauts <law> tagos), 
fragments no iepirkuma dokumentācijas (iekļauts <context> tagos). 
Pārbaudi apgalvojuma patiesumu vai atbildi uz uzdoto jautājumu.
Spriešanai izmanto fragmentu no likumiem, ja tādi ir doti.
Atgriez rezultātu JSON formātā.
1) 'answer' ar iespējamām vērtībām  'jā' vai 'nē'; 
2) 'rate' - pārliecību par atbildi skalā  'zema', 'vidēja', 'augsta'; 
3) 'explanation' - paskaidro savu atbildi, paskaidrojumā iekļauj atsauci uz iepirkuma dokumenta punktiem, citē tos. Ja atbildes gatavošanai trūkst informācija, norādi kas trūkst.

"""

In [53]:
final = prompt
final = final + """

<law>
"""
final = final + law 
final = final + """
</law>
"""
final = final + """
<context>
"""
final = final + context
final = final + """
</context>
"""
final = final + """

pārbaudāmais apgalvojums vai jautājums ir šāds:
"""
final = final + question

#print(final)

response = llm.complete(final)
print(response)

```json
{
  "answer": "nē",
  "rate": "augsta",
  "explanation": "Iepirkuma dokumentācijā (3.6. punkts) ir skaidri norādīts, ka pasūtītājs noraida pretendenta piedāvājumu, ja tiek konstatēti PIL 41. panta vienpadsmitās daļas 2. punktā minētie nosacījumi. Nav minēts, ka tiktu piemērots PIL 41. panta vienpadsmitās daļas 1. punkta regulējums. Tādēļ nav pamata uzskatīt, ka šis regulējums tiktu piemērots."
}
```
